In [1]:
### Installing the required packages if not already installed
packages = ['numpy', 'pandas', 'warnings', 'sqlite3', 'sqlite3', 'scipy']

for package in packages:
    try:
        __import__(package)
    except ImportError:
        %pip install {package}

import numpy as np    # For numerical computing
import pandas as pd   # For data manipulation
import sqlite3        # For connecting to SQL database
import os
from scipy.stats.mstats import winsorize


### Ignoring the warnings
import warnings
warnings.filterwarnings('ignore')

### Pandas display options
pd.options.display.float_format = '{:.4f}'.format

### Setting working directory
os.chdir('/Users/emilwilliamhansen/Desktop/Master-Thesis/Code')

In [2]:
### Extracting the data from the database
conn = sqlite3.connect('Data/data.db')

### Reading the data from the database
stock_ret_d = pd.read_sql('SELECT * FROM filtered_daily_returns', conn).set_index('Date')
stock_ret_m = pd.read_sql('SELECT * FROM filtered_monthly_returns', conn).set_index('Date')
factor_ret_d = pd.read_sql('SELECT * FROM factors_daily', conn).set_index('Date')
factor_ret_m = pd.read_sql('SELECT * FROM factors_monthly', conn).set_index('Date')

### Closing the connection
conn.close()

### Making sure the index is a datetime index
stock_ret_d.index = pd.to_datetime(stock_ret_d.index)
stock_ret_m.index = pd.to_datetime(stock_ret_m.index)
factor_ret_d.index = pd.to_datetime(factor_ret_d.index)
factor_ret_m.index = pd.to_datetime(factor_ret_m.index)

In [3]:
### Winsorizing the monthly returns to inclide 95% of the data
stock_ret_m = stock_ret_m.apply(lambda x: x.clip(lower=x.quantile(0.025), upper=x.quantile(0.975)), axis=1)

In [4]:
stock_ret_m = stock_ret_m.replace(0, np.nan)

In [5]:
def momentum(
        returns: pd.DataFrame,
        lookback_period: int = 12,
        lag: int = 1,
        half_life: float = 2.0,
        n_ptf: int = 5,
        weighting: str = 'ranked'
) -> pd.DataFrame:
    
    exp_w = np.exp(-np.log(2) / half_life * np.arange(lookback_period)[::-1])
    if lag > 0:
        exp_w[-lag:] = 0
    exp_w /= exp_w.sum()

    log_ret = np.log(1 + returns)

    mom_score = log_ret.rolling(lookback_period, min_periods=lookback_period, closed='left').apply(lambda x: np.dot(exp_w, x))

    mom_rank = mom_score.rank(axis=1)

    mom_group = mom_rank.dropna(how='all', axis=0).apply(lambda x: pd.qcut(x, n_ptf, labels=False), axis=1)
    mom_group = mom_group.reindex(returns.index)

    if weighting == 'ranked':
        low_mom_w = mom_rank.where(mom_group == 0, np.nan).apply(lambda x: x / x.abs().sum(), axis=1)
        high_mom_w = mom_rank.where(mom_group == n_ptf-1, np.nan).apply(lambda x: x / x.abs().sum(), axis=1)

    elif weighting == 'equal':
        low_mom_w = mom_rank.where(mom_group == 0, np.nan).apply(lambda x: x / x.abs().sum(), axis=1)
        high_mom_w = mom_rank.where(mom_group == n_ptf-1, np.nan).apply(lambda x: x / x.abs().sum(), axis=1)

        low_mom_w = mom_rank.where(low_mom_w.isna(), 1).apply(lambda x: x / x.abs().sum(), axis=1)
        high_mom_w = mom_rank.where(high_mom_w.isna(), 1).apply(lambda x: x / x.abs().sum(), axis=1)


    exc_ret_mom_H_m = (high_mom_w * stock_ret_m).sum(axis=1)
    exc_ret_mom_L_m = (low_mom_w * stock_ret_m).sum(axis=1)
    exc_ret_mom_m = exc_ret_mom_H_m - exc_ret_mom_L_m

    return exc_ret_mom_m, exc_ret_mom_H_m, exc_ret_mom_L_m, high_mom_w, low_mom_w

In [6]:
fun = momentum(stock_ret_m, lookback_period=12, lag=1, half_life=2, n_ptf=4, weighting='ranked')
mom = fun[0].replace(0, np.nan)
high = fun[1]
low = fun[2]
weights = fun[3].fillna(-fun[4])

In [7]:
### printing the sharpe ratio of the momentum strategy
print(f'The Sharpe ratio of the momentum strategy is {mom.mean() / mom.std()*np.sqrt(12):.2f}')
print(f'The Sharpe ratio of the momentum strategy (high) is {high.mean() / high.std()*np.sqrt(12):.2f}')
print(f'The Sharpe ratio of the momentum strategy (low) is {low.mean() / low.std()*np.sqrt(12):.2f}')

The Sharpe ratio of the momentum strategy is 1.04
The Sharpe ratio of the momentum strategy (high) is 1.01
The Sharpe ratio of the momentum strategy (low) is 0.13


In [8]:
### Using monthly weights to get the daily weights
daily_weights = pd.DataFrame(index=stock_ret_d.index, columns=stock_ret_d.columns)
for date in weights.index:
    short_date = date.strftime('%Y-%m')
    daily_weights.loc[short_date] = weights.loc[date].values

daily_mom = (daily_weights * stock_ret_d).sum(axis=1).replace(0, np.nan)
daily_mom

Date
1980-01-03       NaN
1980-01-04       NaN
1980-01-07       NaN
1980-01-08       NaN
1980-01-09       NaN
               ...  
2023-12-21    0.0057
2023-12-22   -0.0162
2023-12-27   -0.0077
2023-12-28    0.0070
2023-12-29   -0.0011
Length: 11041, dtype: float64

In [10]:
factor_ret_m['UMD'] = mom
factor_ret_d['UMD'] = daily_mom

### Saving the data to the database
conn = sqlite3.connect('Data/data.db')

factor_ret_m.to_sql('factors_monthly', conn, if_exists='replace')
factor_ret_d.to_sql('factors_daily', conn, if_exists='replace')

conn.close()